In [1]:
import pandas as pd

In [2]:
pm_inv = pd.read_csv('../data/output csv/pm_inventory_simplified.csv')

In [3]:
genera_csv = pd.read_csv('../data/output csv/phyto_genera.csv')
genera_csv.head()

,Unnamed: 0,genera,air,cd,chlor,cr,cu,ni,pb,pesticides,petrol,zn,known_uses_count
0,0,Acer,0,0,1,0,0,0,0,0,0,0,1
1,1,Achillea,0,1,0,0,0,0,0,0,0,0,1
2,2,Ambrosia,0,0,0,0,0,0,1,0,0,0,1
3,3,Amorpha,0,0,0,0,1,0,1,0,0,0,2
4,4,Andropogon,0,0,0,0,1,0,0,1,1,0,3


In [4]:
phyto_species_csv = pd.read_csv('../data/output csv/phyto_species.csv')
phyto_species_csv.head()

,Unnamed: 0,genus,species,common name,air,cd,chlor,cr,cu,ni,pb,pesticides,petrol,zn,known_uses_count
0,0,Acer,Acer saccharinum,silver maple,NaN,NaN,chlor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,1,Achillea,Achillea millefolium,common yarrow,NaN,cd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,2,Ambrosia,Ambrosia artemisiifolia,annual ragweed,NaN,NaN,NaN,NaN,NaN,NaN,pb,NaN,NaN,NaN,1
3,3,Amorpha,Amorpha fruticosa,false indigo bush,NaN,NaN,NaN,NaN,cu,NaN,pb,NaN,NaN,NaN,2
4,4,Andropogon,Andropogon gerardii,big bluestem,NaN,NaN,NaN,NaN,cu,NaN,NaN,pest,petrol,NaN,3


In [5]:
plant_type = pm_inv[['genus','plant_type']].drop_duplicates()
plant_type.head()

,genus,plant_type
0,Acorus,Forbs
1,Actaea,Forbs
2,Actaea,Forbs
3,Agalinis,Forbs
5,Agalinis,Forbs


In [6]:
gen_count = pd.merge(genera_csv, plant_type, left_on = 'genera', right_on = 'genus', how = 'left')
gen_count = gen_count[['genera','air','cd','chlor','cr','cu','ni','pb','pesticides','petrol','zn','known_uses_count', 'plant_type']]
gen_count.head()

,genera,air,cd,chlor,cr,cu,ni,pb,pesticides,petrol,zn,known_uses_count,plant_type
0,Acer,0,0,1,0,0,0,0,0,0,0,1,NaN
1,Achillea,0,1,0,0,0,0,0,0,0,0,1,NaN
2,Ambrosia,0,0,0,0,0,0,1,0,0,0,1,NaN
3,Amorpha,0,0,0,0,1,0,1,0,0,0,2,Shrubs-Vines
4,Andropogon,0,0,0,0,1,0,0,1,1,0,3,Grasses-Sedges-Rushes


In [7]:
#dictionary of genera and species with air Particulate Matter remediation
air_df = phyto_species_csv.dropna(subset = ['air'])[['genus','species']]
air_dict = air_df[['genus','species']].set_index('genus').to_dict('dict')['species']

In [8]:
#dictionary of genera and species with heavy metal remediation
hm_df = phyto_species_csv.dropna(subset = ['cd','cr','cu','ni','pb','zn'], how = 'all')[['genus','species','cd','cr','cu','ni','pb','zn']]
hm_dict = hm_df[['genus','species']].set_index('genus').to_dict('dict')['species']

In [9]:
#dictionary of genera and species with lead remediation - included within HM and separate for special value
pb_df = phyto_species_csv[['genus','species','pb']].dropna(subset = 'pb')
pb_dict = pb_df[['genus','species']].set_index('genus').to_dict('dict')['species']

In [10]:
#chlorinated solvents
chlor_df = phyto_species_csv.dropna(subset = ['chlor'])[['genus','species']]
chlor_dict = chlor_df[['genus','species']].set_index('genus').to_dict('dict')['species']

In [11]:
#pesticides
pest_df = phyto_species_csv.dropna(subset = ['pesticides'])[['genus','species']]
pest_dict = pest_df[['genus','species']].set_index('genus').to_dict('dict')['species']

In [12]:
#petroleum
petrol_df = phyto_species_csv.dropna(subset = ['petrol'])[['genus','species']]
petrol_dict = petrol_df[['genus','species']].set_index('genus').to_dict('dict')['species']

In [13]:
#vinyl chloride, manually pulled/separated from PDF since VC is of high interest
vc_dict = {'Acer':'Acer saccharinum', 'Betula':'Betula nigra', 'Cercis':'Cercis canadensis', 'Quercus':'Quercus palustris'}
vc_df = pd.DataFrame(vc_dict.items(), columns = ['genus','species'])

In [14]:
pm_inv['air'] = ''
pm_inv['heavy_metals'] = ''
pm_inv['chlor_solv'] = ''
pm_inv['lead(pb)'] = ''
pm_inv['pesticides'] = ''
pm_inv['petrol'] = ''

In [15]:
#add species and/or genus abilities to Prairie Moon Inventory table

for row_value, row_index in pm_inv.iterrows():
    
    if row_index['genus_species'] in air_dict.values():
        pm_inv.at[row_value, 'air'] = 'air'
    elif row_index['genus'] in air_dict.keys():
        pm_inv.at[row_value, 'air'] = 'air(genus)'

    if row_index['genus_species'] in hm_dict.values():
        pm_inv.at[row_value, 'heavy_metals'] = 'hm'
    elif row_index['genus'] in hm_dict.keys():
        pm_inv.at[row_value, 'heavy_metals'] = 'hm(genus)'

    if row_index['genus_species'] in chlor_dict.values():
        pm_inv.at[row_value, 'chlor_solv'] = 'chlor'
    elif row_index['genus'] in chlor_dict.keys():
        pm_inv.at[row_value, 'chlor_solv'] = 'chlor(genus)'
            
    if row_index['genus_species'] in pb_dict.values():
        pm_inv.at[row_value, 'lead(pb)'] = 'pb'
    elif row_index['genus'] in pb_dict.keys():
        pm_inv.at[row_value, 'lead(pb)'] = 'pb(genus)'
        
    if row_index['genus_species'] in pest_dict.values():
        pm_inv.at[row_value, 'pesticides'] = 'pest'
    elif row_index['genus'] in pest_dict.keys():
        pm_inv.at[row_value, 'pesticides'] = 'pest(genus)'
        
    if row_index['genus_species'] in petrol_dict.values():
        pm_inv.at[row_value, 'petrol'] = 'petrol'
    elif row_index['genus'] in petrol_dict.keys():
        pm_inv.at[row_value, 'petrol'] = 'petrol(genus)'

In [28]:
pm_inv

,genus,genus_species,common_name,plant_type,air,heavy_metals,chlor_solv,lead(pb),pesticides,petrol
0,Acorus,Acorus americanus,Sweet Flag,Forbs,,,,,,
1,Actaea,Actaea pachypoda,Doll’s Eyes,Forbs,,,,,,
2,Actaea,Actaea rubra,Red Baneberry,Forbs,,,,,,
3,Agalinis,Agalinis aspera,Rough False Foxglove,Forbs,,,,,,
4,Agalinis,Agalinis auriculata,Ear-leaf False Foxglove,Forbs,,,,,,
...,...,...,...,...,...,...,...,...,...,...
560,Sambucus,Sambucus canadensis,Elderberry,Shrubs-Vines,,,,,,
561,Spiraea,Spiraea tomentosa,Steeplebush,Shrubs-Vines,,,,,,
562,Symphoricarpos,Symphoricarpos albus,Snowberry,Shrubs-Vines,,,,,,
563,Symphoricarpos,Symphoricarpos orbiculatus,Coralberry,Shrubs-Vines,,,,,,


In [16]:
#create dictionary - {pollutant type : site type}
pollutant_dict = {'air':['roadway','railroad','industrial site','urban, suburban, other residential area','wildfire'],
                 'chlor':['railroad','industrial site','dry cleaner','autobody shop'],
                 'HM':['industrial site','tannery','smelting operation','agricultural area','building older than 1978'],
                 'Pb':['building older than 1978','roadway'],
                 'pest':['railroad','urban, suburban, other residential area','agricultural area','golf course','orchard','utility corridor'],
                 'petrol':['railroad','urban, suburban, other residential area','autobody shop','parking lot or driveway']}    

In [17]:
sites_list = ['agricultural area', 'autobody shop', 'building older than 1978', 'dry cleaner', 'golf course', 
              'industrial site', 'orchard', 'parking lot or driveway','railroad', 'roadway', 'smelting operation',
             'tannery', 'urban, suburban, other residential area', 'utility corridor', 'wildfire']

In [18]:
sites_df = pd.DataFrame(sites_list, columns = ['site use (current or historical)'])
sites_df = sites_df.assign(air = '', chlor = '', HM = '', Pb = '', pest = '', petrol = '')

In [19]:
#site - potential pollutants
for row_value, row_index in sites_df.iterrows():
    if row_index['site use (current or historical)'] in pollutant_dict['air']:
        sites_df.at[row_value, 'air'] = 'air'
    
    if row_index['site use (current or historical)'] in pollutant_dict['chlor']:
        sites_df.at[row_value, 'chlor'] = 'chlor'

    if row_index['site use (current or historical)'] in pollutant_dict['HM']:
        sites_df.at[row_value, 'HM'] = 'HM'
        
    if row_index['site use (current or historical)'] in pollutant_dict['Pb']:
        sites_df.at[row_value, 'Pb'] = 'Pb'
           
    if row_index['site use (current or historical)'] in pollutant_dict['pest']:
        sites_df.at[row_value, 'pest'] = 'pest'
            
    if row_index['site use (current or historical)'] in pollutant_dict['petrol']:
        sites_df.at[row_value, 'petrol'] = 'petrol'
        
sites_df                 

,site use (current or historical),air,chlor,HM,Pb,pest,petrol
0,agricultural area,,,HM,,pest,
1,autobody shop,,chlor,,,,petrol
2,building older than 1978,,,HM,Pb,,
3,dry cleaner,,chlor,,,,
4,golf course,,,,,pest,
5,industrial site,air,chlor,HM,,,
6,orchard,,,,,pest,
7,parking lot or driveway,,,,,,petrol
8,railroad,air,chlor,,,pest,petrol
9,roadway,air,,,Pb,,


In [47]:
site_dict = {'agriculture':['','','HM','','pest',''], 'autobody shop':['','chlor','','','','petrol'], 'old building':['','','HM','Pb','',''], 
             'dry cleaner':['','chlor','','','',''], 'golf course':['','','','','pest',''], 'industrial site':['air','chlor','HM', '','',''], 
             'orchard':['','','','','pest',''], 'parking lot':['','','','','','petrol'],'railroad':['air','chlor','','','pest','petrol'],
             'roadway':['','','HM','Pb','','petrol'], 'smelting':['','','HM','','',''],'tannery':['','','HM','','',''], 
             'residential':['air','','','','pest','petrol'], 'utility':['','','','','pest',''], 'wildfire':['air','','','','','']}

In [48]:
site_df2 = pd.DataFrame(site_dict)

In [49]:
site_df2 = site_df2.assign(pollutant = pollutant_dict.keys())

In [50]:
site_df2 = site_df2[['pollutant', 'agriculture', 'autobody shop', 'old building', 'dry cleaner',
       'golf course', 'industrial site', 'orchard', 'parking lot', 'railroad',
       'roadway', 'smelting', 'tannery', 'residential', 'utility', 'wildfire']]

In [51]:
site_df2

,pollutant,agriculture,autobody shop,old building,dry cleaner,golf course,industrial site,orchard,parking lot,railroad,roadway,smelting,tannery,residential,utility,wildfire
0,air,,,,,,air,,,air,,,,air,,air
1,chlor,,chlor,,chlor,,chlor,,,chlor,,,,,,
2,HM,HM,,HM,,,HM,,,,HM,HM,HM,,,
3,Pb,,,Pb,,,,,,,Pb,,,,,
4,pest,pest,,,,pest,,pest,,pest,,,,pest,pest,
5,petrol,,petrol,,,,,,petrol,petrol,petrol,,,petrol,,


In [41]:
ag_top = pm_inv.loc[(pm_inv['heavy_metals'].str.contains('hm')) & (pm_inv['pesticides'].str.contains('pest'))]
ag_sec = pm_inv.loc[(pm_inv['heavy_metals'].str.contains('hm')) | (pm_inv['pesticides'].str.contains('pest'))]

In [45]:
auto_top = pm_inv.loc[(pm_inv['chlor_solv'].str.contains('chlor')) & (pm_inv['petrol'].str.contains('pet'))]
auto_sec = pm_inv.loc[(pm_inv['chlor_solv'].str.contains('chlor')) | (pm_inv['petrol'].str.contains('pet'))]


,genus,genus_species,common_name,plant_type,air,heavy_metals,chlor_solv,lead(pb),pesticides,petrol
172,Helianthus,Helianthus angustifolius,Swamp Sunflower,Forbs,,hm(genus),,,,petrol(genus)
173,Helianthus,Helianthus giganteus,Tall Sunflower,Forbs,,hm(genus),,,,petrol(genus)
174,Helianthus,Helianthus grosseserratus,Saw-tooth Sunflower,Forbs,,hm(genus),,,,petrol(genus)
175,Helianthus,Helianthus maximiliani,Maximilian’s Sunflower,Forbs,,hm(genus),,,,petrol(genus)
176,Helianthus,Helianthus mollis,Downy Sunflower,Forbs,,hm(genus),,,,petrol(genus)
...,...,...,...,...,...,...,...,...,...,...
526,Scirpus,Scirpus pendulus,Rufous Bulrush,Grasses-Sedges-Rushes,,,,,,petrol
527,Scirpus,Scirpus pungens,Chairmaker’S Rush,Grasses-Sedges-Rushes,,,,,,petrol(genus)
528,Scirpus,Scirpus validus,Great Bulrush,Grasses-Sedges-Rushes,,,,,,petrol(genus)
530,Sorghastrum,Sorghastrum nutans,Indian Grass Pls,Grasses-Sedges-Rushes,,,,,pest,petrol


In [53]:
# pm_inv.to_csv('../data/output csv/pm_inv_uses.csv', header = True, index = False)
# air_df.to_csv('../data/output csv/air_trees.csv', header = True, index = False)
# vc_df.to_csv('../data/output csv/vc_trees.csv', header = True, index = False)
# sites_df.to_csv('../data/output csv/site_pollutants.csv', header = True, index = False)
# site_df2.to_csv('../data/output csv/sites_df2.csv', header = True, index = False)

In [29]:
pm_inv.head()

,genus,genus_species,common_name,plant_type,air,heavy_metals,chlor_solv,lead(pb),pesticides,petrol
0,Acorus,Acorus americanus,Sweet Flag,Forbs,,,,,,
1,Actaea,Actaea pachypoda,Doll’s Eyes,Forbs,,,,,,
2,Actaea,Actaea rubra,Red Baneberry,Forbs,,,,,,
3,Agalinis,Agalinis aspera,Rough False Foxglove,Forbs,,,,,,
4,Agalinis,Agalinis auriculata,Ear-leaf False Foxglove,Forbs,,,,,,
